In [8]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore", message="Thread 'MainThread': missing ScriptRunContext!")
from sklearn.metrics.pairwise import cosine_similarity
import streamlit as st

In [9]:
# Load the dataset
rating_matrix = pd.read_csv("I-w9Wo-HSzmUGNNHw0pCzg_bc290b0e6b3a45c19f62b1b82b1699f1_Rmat.csv", index_col=0)

# System I: Recommendation Based on Popularity

In [17]:
def recommend_popular(rating_matrix, top_n=10):
    # Calculate popularity (e.g., average rating * number of ratings)
    movie_popularity = rating_matrix.apply(lambda col: col.mean(skipna=True) * col.notna().sum(), axis=0)
    top_movies = movie_popularity.sort_values(ascending=False).head(top_n)
    
    return top_movies.index.tolist(), top_movies.values

In [18]:
# Normalize rows by subtracting the mean
def normalize_matrix(matrix):
    row_means = matrix.mean(axis=1, skipna=True)
    
    return matrix.sub(row_means, axis=0)

In [19]:
def compute_cosine_similarity_optimized(matrix):
    # Transpose the matrix to compute similarities between columns (movies)
    movie_features = matrix.fillna(0).to_numpy().T
    
    # Use sklearn's cosine_similarity
    cosine_sim = cosine_similarity(movie_features)
    
    # Transform similarity to (1 + cos) / 2
    similarity = (1 + cosine_sim) / 2
    
    # Mask similarities for less than 3 users rating both movies
    user_counts = (~matrix.isna()).T.dot(~matrix.isna())
    similarity[user_counts < 3] = np.nan
    
    return pd.DataFrame(similarity, index=matrix.columns, columns=matrix.columns)

# System II: IBCF-based Recommendations

In [20]:
def myIBCF(new_user_ratings, similarity_matrix, rating_matrix):
    predictions = {}
    
    for movie in similarity_matrix.index:
        if pd.isna(new_user_ratings[movie]):
            related_movies = similarity_matrix.loc[movie].dropna()
            rated_movies = new_user_ratings[~new_user_ratings.isna()]
            relevant_movies = related_movies.index.intersection(rated_movies.index)
            
            if relevant_movies.any():
                weights = related_movies.loc[relevant_movies]
                ratings = rated_movies.loc[relevant_movies]
                prediction = (weights * ratings).sum() / weights.sum()
                predictions[movie] = prediction
    
    # Sort by predicted ratings
    sorted_predictions = sorted(predictions.items(), key=lambda x: x[1], reverse=True)
    return [movie for movie, _ in sorted_predictions[:10]]

# Test the implementation

In [22]:
# Test the implementation with user u1181
if __name__ == "__main__":
    # Popularity-based recommendations
    top_movies, scores = recommend_popular(rating_matrix)
    print("Top 10 Popular Movies:", top_movies)

    # Normalize matrix and compute similarity
    normalized_matrix = normalize_matrix(rating_matrix)
    similarity_matrix = cosine_similarity(normalized_matrix)

    # User "u1181" input from the rating matrix
    user_ratings_u1181 = rating_matrix.loc["u1181"]

    # IBCF-based recommendations for u1181
    ibcf_recommendations_u1181 = myIBCF(user_ratings_u1181, similarity_matrix, rating_matrix)
    print("Top 10 IBCF Recommendations for u1181:", ibcf_recommendations_u1181)

    # Hypothetical user input
    user_ratings_hypothetical = pd.Series(index=rating_matrix.columns, dtype="float")
    user_ratings_hypothetical["m1613"] = 5
    user_ratings_hypothetical["m1755"] = 4

    # IBCF-based recommendations for hypothetical user
    ibcf_recommendations_hypothetical = myIBCF(user_ratings_hypothetical, similarity_matrix, rating_matrix)
    print("Top 10 IBCF Recommendations for hypothetical user:", ibcf_recommendations_hypothetical)

Top 10 Popular Movies: ['m2858', 'm260', 'm1196', 'm1210', 'm2028', 'm1198', 'm593', 'm2571', 'm2762', 'm589']


ValueError: Input contains NaN.

# The App


In [16]:
def compute_similarity_matrix(rating_matrix):
    # Fill missing values with 0 for cosine similarity calculation
    filled_matrix = rating_matrix.fillna(0)
    # Compute cosine similarity
    similarity = cosine_similarity(filled_matrix.T)
    # Convert to DataFrame
    similarity_df = pd.DataFrame(similarity, index=rating_matrix.columns, columns=rating_matrix.columns)
    return similarity_df

# Load the rating matrix
rating_matrix = pd.read_csv("I-w9Wo-HSzmUGNNHw0pCzg_bc290b0e6b3a45c19f62b1b82b1699f1_Rmat.csv", index_col=0)

# Compute similarity matrix
similarity_matrix = compute_similarity_matrix(rating_matrix)

# Save to CSV
similarity_matrix.to_csv("similarity_matrix.csv")


In [21]:
# Load Data
@st.cache_data
def load_data():
    # Load the rating matrix and similarity matrix
    rating_matrix = pd.read_csv("I-w9Wo-HSzmUGNNHw0pCzg_bc290b0e6b3a45c19f62b1b82b1699f1_Rmat.csv", index_col=0)
    similarity_matrix = pd.read_csv("similarity_matrix.csv", index_col=0)
    return rating_matrix, similarity_matrix

# Normalize matrix
def normalize_matrix(matrix):
    row_means = matrix.mean(axis=1, skipna=True)
    return matrix.sub(row_means, axis=0)

# Recommendation function
def myIBCF(new_user_ratings, similarity_matrix):
    predictions = {}
    for movie in similarity_matrix.index:
        if pd.isna(new_user_ratings.get(movie, np.nan)):  # Only predict for unrated movies
            similar_movies = similarity_matrix[movie].dropna()
            rated_movies = new_user_ratings.dropna()
            relevant_movies = rated_movies.index.intersection(similar_movies.index)

            if len(relevant_movies) > 0:
                weights = similar_movies[relevant_movies]
                ratings = rated_movies[relevant_movies]
                denominator = weights.sum()
                numerator = (weights * ratings).sum()

                if denominator > 0:
                    predictions[movie] = numerator / denominator
    sorted_predictions = sorted(predictions.items(), key=lambda x: x[1], reverse=True)
    return [movie for movie, _ in sorted_predictions[:10]]

# Main App
def main():
    st.title("Movie Recommender System")
    st.subheader("Rate Movies to Get Personalized Recommendations")

    # Load data
    rating_matrix, similarity_matrix = load_data()

    # Show sample movies to rate
    sample_movies = rating_matrix.columns[:100]
    st.write("Please rate the following sample movies (1-5 stars or leave blank):")
    user_ratings = {}
    for movie in sample_movies:
        user_ratings[movie] = st.slider(f"{movie}", min_value=1, max_value=5, value=None, step=1, format="%d")

    # Convert user ratings to a Pandas Series
    user_ratings_series = pd.Series(user_ratings).dropna()

    # Show recommendations if ratings are provided
    if st.button("Get Recommendations"):
        if user_ratings_series.empty:
            st.warning("Please rate at least one movie to get recommendations.")
        else:
            recommendations = myIBCF(user_ratings_series, similarity_matrix)
            st.success("Top 10 Recommended Movies for You:")
            for movie in recommendations:
                st.write(movie)

2024-12-16 18:30:30.917 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


2024-12-10 22:03:43.153 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.154 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.155 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.156 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.156 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.161 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.162 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.162 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

2024-12-10 22:03:43.213 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.214 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.215 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.216 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.217 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.217 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.219 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.220 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

2024-12-10 22:03:43.274 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.275 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.276 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.277 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.277 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.279 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.280 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.280 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

2024-12-10 22:03:43.341 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.341 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.342 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.343 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.343 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.344 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.344 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.345 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

2024-12-10 22:03:43.405 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.405 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.406 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.407 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.407 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.409 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.409 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.410 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

2024-12-10 22:03:43.460 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.460 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.461 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.462 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.462 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.465 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.466 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.466 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

2024-12-10 22:03:43.513 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.514 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.515 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.515 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.516 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.516 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.517 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.517 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

2024-12-10 22:03:43.553 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.553 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.554 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.554 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.555 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.555 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.556 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-10 22:03:43.556 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [23]:
if __name__ == "__main__":
    main()

2024-12-16 18:30:47.492 
  command:

    streamlit run /opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-12-16 18:30:47.493 No runtime found, using MemoryCacheStorageManager
2024-12-16 18:30:49.964 Session state does not function when running a script without `streamlit run`
